In [2]:

from dotenv import load_dotenv
import os
import sys
import json
import pandas as pd
import time
import datetime
load_dotenv()

path = os.path.abspath('')
module_path = os.path.join(path,'..\..')
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path+"\\functions")


from dataset_utils import DatasetEvaluator

c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\experiments\SimilarPairs\..\..


<>:11: SyntaxWarning: invalid escape sequence '\.'
<>:11: SyntaxWarning: invalid escape sequence '\.'
C:\Users\Tahsin\AppData\Local\Temp\ipykernel_16560\2745404199.py:11: SyntaxWarning: invalid escape sequence '\.'
  module_path = os.path.join(path,'..\..')


In [3]:
DEFAULT_PATH = "results/"
EXPERIMENT_RESULT_PATH = f"{DEFAULT_PATH}sql_query_chain_result_files.txt"
EXPERIMENT = "SQLQueryChain"
TARGET_PATH = f"evaluator_results/{EXPERIMENT}_results.csv"
TARGET_PATH_FK = f"evaluator_results/{EXPERIMENT}_foreign_key_results.csv"


Useful functions

In [4]:
def save_queries(queries, FILE_NAME_PATH):
    data = {"queries":queries}
    with open(FILE_NAME_PATH, "w") as arquivo_json:
        json.dump(data, arquivo_json, indent=4) 

def read_queries(FILE_NAME_PATH):
    with open(FILE_NAME_PATH, encoding="utf-8", errors="ignore") as json_data:
        data = json.load(json_data, strict=False)
    queries = data["queries"]
    return queries

def read_files():
    with open(EXPERIMENT_RESULT_PATH, "r") as f:
        files = f.readlines()
    files = [f"{DEFAULT_PATH}{x.strip()}" for x in files]
    return files

def get_schema(FILE_NAME_PATH):
    if "mondial_gpt" in FILE_NAME_PATH:
        return "mondial_gpt"
    elif "mondial" in FILE_NAME_PATH:
        return "mondial"

    return ""

def get_count_query_type(queries):
    count_types = {}
    for item in queries:
        type = item["type"]
        if type in count_types:
            count_types[type] += 1
        else:
            count_types[type] = 1
    count_types['total'] = len(queries)
    return count_types

def get_dataset_evaluator(FILE_NAME_PATH):
    SCHEMA = get_schema(FILE_NAME_PATH)  
    PREFIX = SCHEMA  
    if SCHEMA == "mondial_gpt":
        PREFIX = "mondial"
    print(PREFIX)
    print(SCHEMA)
    dataset_eval = DatasetEvaluator(dataset_file_path=f"../../datasets/{PREFIX}/{PREFIX}_dataset.json",
                                dataset_tables_path=f"../../datasets/{PREFIX}/result_tables/",
                                db_connection_file=f"../../datasets/{SCHEMA}_db_connection.json",
                                dataset_name=SCHEMA)
    return dataset_eval

def get_files_in_folder():
    files = []
    for r, d, f in os.walk(DEFAULT_PATH):
        for file in f:
            if '.json' in file and 'erro' not in file:
                files.append(os.path.join(r, file))
    return sorted(files)

def evaluate_query(queries, dataset_eval, FILE_NAME_PATH):
    result = {'simple': 0, 'medium': 0, 'complex': 0, 'total': 0}

    for q in queries:
        evaluate_result = False
        q['similarity'] = 0
        try:
            if q["query_string"].strip() != "":
                evaluate_result, similarity, column_matching_index = dataset_eval.evaluate_dataset_query(q["query_string"], q["id"], query_type="sql")
                q['column_matching_index'] = column_matching_index
                q['similarity']=similarity
            
        except Exception as e:
            print(str(e))
            
        q['result'] = evaluate_result
        
        if evaluate_result==True:
            result['total'] += 1
            result[q["type"]] += 1
        
    save_queries(queries, FILE_NAME_PATH)   
    return result

def get_file_name(FILE_NAME_PATH):
    parts = FILE_NAME_PATH.split("/")
    last_element = parts[-1]
    file_name = last_element.replace(".json", "")
    return file_name

def get_experiment_name(filename):
    schema = get_schema(filename).upper()
    experiment_name = f"{EXPERIMENT} - {schema}"
    if "chatgpt4" in filename:
        experiment_name += " - GPT-4"
    elif "chatgpt_" in filename:
        experiment_name += " - GPT-3.5"
    elif "llama" in filename:
        experiment_name += " - LLAMA"
    
    if "rag" in filename:
        experiment_name += " - RAG"
        
    if "turbo" in filename:
        experiment_name += " - Turbo"
                        
    if "instances" in filename:
        experiment_name += " - Passing database instances"
        
    if "_fk" in filename:
        experiment_name += " - FK"
        
    return experiment_name


def compute_cost(filename):
    queries = read_queries(filename)
    total_costs = 0
    total_tokens = 0
    input_tokens = 0
    output_tokens = 0
    for q in queries:
        total_tokens += q.get('total_tokens', 0)
        total_costs += q.get('total_cost', 0)
        input_tokens += q.get('prompt_tokens', 0)
        output_tokens += q.get('completion_tokens', 0)

    print(total_tokens)
    print(round(total_costs, 2))
    print(input_tokens, output_tokens)
    return total_tokens, round(total_costs, 2), input_tokens, output_tokens

def compute_time(filename):
    queries = read_queries(filename)
    total_time = 0
    for q in queries:
        total_time += q.get('time', 0)
    print(total_time)
    return str(datetime.timedelta(seconds=round(total_time)))

In [5]:
def run(result_files = [], result_final = [], compute_costs = True):
    
    if result_files == [] :
        result_files = get_files_in_folder()
    else:
        result_files = sorted(result_files)
        
    
    for file in result_files:
        queries = read_queries(file)
        dataset_eval = get_dataset_evaluator(file)
        count_query_type = get_count_query_type(queries)
        result = evaluate_query(queries, dataset_eval, file)
        
        overall_by_type = {}
        for item in result:
            overall_by_type[f"overall_{item}"] = round(result[item] / count_query_type[item], 4)
        
        result['experiment'] = get_file_name(file)
        result = {**result, **overall_by_type}
        result['total_tokens'], result['total_cost'], result['average_tokens_by_query'] = 0, 0, 0
        if compute_costs:
            result['total_tokens'], result['total_cost'], result['input_tokens'], result['output_tokens'] = compute_cost(file)

        result['total_time'] = compute_time(file)
        
        result_final.append(result)
        time.sleep(5)
    return result_final
        

## Validation with mondial - Foreign Keys

In [6]:
results_fk = run(['sql_queries_chatgpt_few_shot_50_mondial_gpt.json'], [])

mondial
mondial_gpt
{'DB_HOST': 'localhost', 'DB_PORT': '1522', 'DB_USER_NAME': 'MONDIAL_GPT', 'DB_PASS': 'TextDB123', 'DB_NAME': '', 'SQL_DRIVER': 'oracle+oracledb', 'SERVICE_NAME': 'xepdb1', 'SCHEMA': 'MONDIAL_GPT', 'KEYWORD_SEARCH_API_URL': ''}
SELECT c.capital FROM mondial_country c WHERE c.capital NOT IN (SELECT lo.city FROM mondial_located lo )
SELECT c.capital 
FROM country c
LEFT JOIN located l ON c.capital = l.city
WHERE l.city IS NULL


c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\experiments\SimilarPairs\..\..\functions\dataset_utils.py:423: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql(SQL_query, con=self.db_connection)
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\experiments\SimilarPairs\..\..\functions\dataset_utils.py:423: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql(SQL_query, con=self.db_connection)
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\experiments\SimilarPairs\..\..\functions\dataset_utils.py:423: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqli

True cols: Index(['CAPITAL'], dtype='object')
Adj. True cols: Index(['CAPITAL'], dtype='object')
Predicted cols: Index(['CAPITAL'], dtype='object')
Adj. Predicted cols: Index(['CAPITAL'], dtype='object')
True: query matches sql (Similarity: 0.9846153846153847, column match: 1.0)
SELECT p.CAPITAL
FROM MONDIAL_PROVINCE p 
WHERE p.name like '%Andalucía%'
SELECT capital FROM province WHERE name = 'Andalucía'
True cols: Index(['CAPITAL'], dtype='object')
Adj. True cols: Index([], dtype='object')
Predicted cols: Index(['CAPITAL'], dtype='object')
Adj. Predicted cols: Index([], dtype='object')
False: no match found
SELECT abbreviation FROM mondial_organization WHERE name='General Confederation of Trade Unions'
SELECT abbreviation FROM organization WHERE name = 'General Confederation of Trade Unions'
True cols: Index(['ABBREVIATION'], dtype='object')
Adj. True cols: Index(['ABBREVIATION'], dtype='object')
Predicted cols: Index(['ABBREVIATION'], dtype='object')
Adj. Predicted cols: Index(['ABBR

c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\experiments\SimilarPairs\..\..\functions\dataset_utils.py:423: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql(SQL_query, con=self.db_connection)
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\experiments\SimilarPairs\..\..\functions\dataset_utils.py:423: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql(SQL_query, con=self.db_connection)
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\experiments\SimilarPairs\..\..\functions\dataset_utils.py:423: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqli

True cols: Index(['SUM(AREA)'], dtype='object')
Adj. True cols: Index(['SUM(AREA)'], dtype='object')
Predicted cols: Index(['TOTAL_AREA'], dtype='object')
Adj. Predicted cols: Index(['TOTAL_AREA'], dtype='object')
True: query matches sql (Similarity: 1.0, column match: 1.0)
SELECT name
FROM mondial_country
WHERE name LIKE 'B%' AND population > 10000000;
SQLQuery: SELECT name 
FROM country 
WHERE name LIKE 'B%' 
AND population > 10000000
Error: Execution failed on sql 'SQLQuery: SELECT name 
FROM country 
WHERE name LIKE 'B%' 
AND population > 10000000': DPY-4010: a bind variable replacement value for placeholder ":SELECT" was not provided
SELECT cio.othername
FROM mondial_cityothername cio
INNER JOIN mondial_country c
ON cio.country = c.code
WHERE c.name = 'Brazil'
SELECT city.name, citylocalname.localname
FROM city
JOIN citylocalname ON city.name = citylocalname.city AND city.country = citylocalname.country
WHERE city.country = 'BRAZ'
True cols: Index(['OTHERNAME'], dtype='object')
Ad

c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\experiments\SimilarPairs\..\..\functions\dataset_utils.py:423: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql(SQL_query, con=self.db_connection)
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\experiments\SimilarPairs\..\..\functions\dataset_utils.py:423: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql(SQL_query, con=self.db_connection)
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\experiments\SimilarPairs\..\..\functions\dataset_utils.py:423: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqli

In [7]:
df = pd.DataFrame(results_fk, columns=['experiment', 'simple', 'medium', 'complex', 'total', 'overall_simple', 'overall_medium', 'overall_complex', 'overall_total','input_tokens', 'output_tokens', 'total_tokens','total_cost', 'total_time'])
df['experiment'] = df['experiment'].apply(get_experiment_name)
df = df.sort_values(by='overall_total', ascending=False)
df

,experiment,simple,medium,complex,total,overall_simple,overall_medium,overall_complex,overall_total,input_tokens,output_tokens,total_tokens,total_cost,total_time
0,SQLQueryChain - MONDIAL_GPT - GPT-3.5,12,11,9,32,0.7059,0.6471,0.5625,0.64,310284,1531,311815,0.94,0:01:19


In [8]:
df.to_csv("results.csv")  